<h1>Problema 1: Jóvenes/adultos</h1>
<p>Autor: Erick García Ramírez<br /> 
erick_phy@ciencias.unam.mx<br /> 
Curso de Aprendizaje Automatizado, MCIC 2019-2</p>

En este problema pretendemos clasificar a individuos como jóvenes o adultos a partir de sus gustos musicales. El ejericio tiene el objetivo de comparar a un clasificador bayesiano ingenuo, mve_classsifierNB, construido a partir del principio de máxima verosimilitud contra otro, map_classifierNB, construido a partir del principio de máximo a posteriori. Mostraremos que map_classifierNB logra clasificar a individuos que mve_classifier no logra clasificar.

Ambos clasificadores serán construidos directamente, aunque evitaremos hacer manualmente las cuentas (por ejemplo, como hay 6 bandas, frecuentemente requerimos hacer productos de al menos 6 factores; automatizamos tales cálculos con rutinas sencillas). 

In [28]:
import pandas as pd
import numpy as np

En la siguiente celda cargamos y preparamos los datos el problema.

In [29]:
# Datos de entrenamiento
data = np.array([
            ['','Pink Floyd','The Beatles','R.E.M.','Nirvana','Queen','Oasis','Class'],
            ['P1', 1, 0, 0, 1, 1, 1, 'J'],
            ['P2', 1, 1, 0, 1, 1, 0, 'J'],
            ['P3', 1, 1, 1, 0, 0, 1, 'J'],
            ['P4', 1, 0, 1, 0, 0, 1, 'J'],
            ['P5', 1, 0, 0, 0, 1, 0, 'J'],
            ['P6', 1, 1, 1, 0, 0, 0, 'J'],
            ['P7', 1, 1, 0, 0, 1, 1, 'A'],
            ['P8', 1, 1, 1, 0, 0, 1, 'A'],
            ['P9', 1, 1, 1, 1, 1, 0, 'A'],
            ['P10',1, 1, 1, 0, 1, 0, 'A'],
            ['P11',1, 1, 1, 0, 1, 1, 'A'],
            ['P12',1, 1, 0, 1, 1, 0, 'A'],
            ['P13',1, 1, 0, 1, 0, 0, 'A']])
    
dF=pd.DataFrame(data=data[1:,1:],
                  index=data[1:,0],
                  columns=data[0,1:])
dF

,Pink Floyd,The Beatles,R.E.M.,Nirvana,Queen,Oasis,Class
P1,1,0,0,1,1,1,J
P2,1,1,0,1,1,0,J
P3,1,1,1,0,0,1,J
P4,1,0,1,0,0,1,J
P5,1,0,0,0,1,0,J
P6,1,1,1,0,0,0,J
P7,1,1,0,0,1,1,A
P8,1,1,1,0,0,1,A
P9,1,1,1,1,1,0,A
P10,1,1,1,0,1,0,A


Recordemos que en general,

$$P(C=c|\chi)\propto P(C=c)\Pi_{t=1}^{|\chi|}P(\chi^{t}=x^{(t)}|C=c)$$
Por lo que debemos estimar los parámetros para $P(C=c)$ y $P(\chi^{t}=x^{t}|C=c)$, para cada $t=1,2,3,4,5,6$.

 Primer hacemos la estimación de dichos parámetros bajo máxima verosimilitud. 

In [30]:
# Cálculo de P(band|clase) 
def prob_of_band_given_class(band,cl):
    Nc=0
    nc_band=0
    for k in dF.index:
        if dF.loc[k]['Class'] == cl:
            Nc += 1
            if dF.loc[k][band] == '1':
                nc_band += 1
    return nc_band/Nc

# Cálculo de P(clase)
def prob_of_class(cl):
    counter=0
    for k in dF.index:
        if dF.loc[k]['Class'] == cl:
            counter +=1
    return counter/len(dF.index)

# Cálculo de P(clase|persona) con MVE, la probabilidad de que una persona dada pertenezca a la clase
def mve_prob_of_being_of_class(cl,person):
    p=0
    p= prob_of_class(cl)
    for k in dF.columns[:-1]:
        if person[k] == '1':
            p *= prob_of_band_given_class(k,cl)
        else:
            p *= (1-prob_of_band_given_class(k,cl))
    return p

<h2> El clasificador mve_classifierNB </h2>
Con los parámetros arriba estimados, podemos ahora construir el clasificador.

In [31]:
# Clasificador bayesiano ingenuo MEV. Clasifica un individuo como 'A' (adulto) o 'J' (joven)
def mve_classifierNB(person):
    if mve_prob_of_being_of_class('A',person) < mve_prob_of_being_of_class('J',person):
        return 'J'
    if mve_prob_of_being_of_class('J',person) < mve_prob_of_being_of_class('A',person):
        return 'A'

<h2> Predicciones con mve_classifierNB</h2>
Procedemos a presentar las predicciones para los nuevos individuos con el clasificador construido hasta ahora. Podemos observar que el individuo x5 no puede clasificado por mve_classifierNB. Esto se debe a que el clasificador encuentra que ambas probabilidades de ser joven o adulto son iguales a 0.0. 

In [32]:
# Datos para predicciones
tdata = np.array([    
        ['','Pink Floyd','The Beatles', 'R.E.M.','Nirvana','Queen','Oasis'],
        ['x1',1, 1, 0, 1, 1, 0],
        ['x2',1, 0, 1, 1, 1, 1],
        ['x3',1, 1, 0, 0, 0, 0],
        ['x4',1, 1, 1, 1, 1, 1],
        ['x5',0, 1, 1, 1, 1, 1]])

testdF=pd.DataFrame(data=tdata[1:,1:],
                  index=tdata[1:,0],
                  columns=tdata[0,1:])
testdF

,Pink Floyd,The Beatles,R.E.M.,Nirvana,Queen,Oasis
x1,1,1,0,1,1,0
x2,1,0,1,1,1,1
x3,1,1,0,0,0,0
x4,1,1,1,1,1,1
x5,0,1,1,1,1,1


In [33]:
# Predicciones con 
for k in testdF.index:
    print(mve_classifierNB(testdF.loc[k]))

A
J
A
A
None


Ahora estimamos parámetros siguiendo máximo a posteiori.

In [34]:
# Suponemos que P(C) tiene distribución Bernoulli B(q). Buscamos q.

def qmap_class(cl):
    Nc=0
    for k in dF.index:
        if dF.loc[k]['Class'] == cl:
            Nc += 1
    return (Nc+1)/(8+Nc)

def qmap_band_given_class(cl,band):
    nc_a = 0
    Nc = 0
    for k in dF.index:
        if dF.loc[k]['Class'] == cl:
            Nc +=1
            if dF.loc[k][band] == '1':
                nc_a +=1 
    return (nc_a +1)/(6+Nc)

def qmaps_given_class(cl):
    qmaps={}
    for k in dF.columns[:-1]:
        qmaps[k] = qmap_band_given_class(cl,k)
    return qmaps

# Cálculo de P(clase|persona)
def map_prob_of_being_of_class(cl,person):
    qmaps = qmaps_given_class(cl)
    p = qmap_class(cl)
    for k in dF.columns[:-1]:
        if person[k] == '1':
            p *= qmaps[k]
        else: 
            p *= (1-qmaps[k])
    return p

<h2>El clasificador map_classifierNB </h2>
Ahora podemos construir el clasificador.

In [35]:
def map_classifierNB(person):
    if map_prob_of_being_of_class('A',person) < map_prob_of_being_of_class('J',person):
        return 'J'
    if map_prob_of_being_of_class('J',person) < map_prob_of_being_of_class('A',person):
        return 'A'

<h2>Predicciones y conclusiones</h2>
Debajo tenemos la lista de el rendimiento de los dos clasificadores. Observe que map_classifierNB logra clasificar a x5 como adulto. Esto se logra gracias al suavizado introducido bajo máximo a posteriori (las probabilidades no se colapsan a 0). Compare las probabilidades respectivas para x5.

In [36]:
print('Per','OCl','MVE', 'MAP')
for k in dF.index:
    print(k,' ', dF.loc[k]['Class'],' ', mve_classifierNB(dF.loc[k]),' ',map_classifierNB(dF.loc[k]))
for k in testdF.index:
    print(k,' ', 'UNK',' ', mve_classifierNB(testdF.loc[k]),' ',map_classifierNB(testdF.loc[k]))

Per OCl MVE MAP
P1   J   J   J
P2   J   A   A
P3   J   A   A
P4   J   J   J
P5   J   J   J
P6   J   A   A
P7   A   A   A
P8   A   A   A
P9   A   A   A
P10   A   A   A
P11   A   A   A
P12   A   A   A
P13   A   A   A
x1   UNK   A   A
x2   UNK   J   A
x3   UNK   A   A
x4   UNK   A   A
x5   UNK   None   A


In [37]:
print(mve_prob_of_being_of_class('A',testdF.loc['x5']),mve_prob_of_being_of_class('J',testdF.loc['x5']))
print(map_prob_of_being_of_class('A',testdF.loc['x5']),map_prob_of_being_of_class('J',testdF.loc['x5']))

0.0 0.0
0.0021214844009779547 0.0006430041152263372
